In [18]:
# !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from urllib.request import urlopen
import os
import urllib.request
import numpy as np
import cv2 


In [2]:
URL = "http://image-net.org/index"

options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome('./chromedriver.exe', options=options)
wait = WebDriverWait(driver, 20)

# Word to search
keyword = input("Enter a keyword")
driver.get(URL)

# Get the input and search the word with submit
search = driver.find_element_by_name("q")
search.send_keys(keyword)
search.submit()

res = wait.until(presence_of_element_located((By.CSS_SELECTOR, "span.result_synset")))
res.click()

download_tab = wait.until(presence_of_element_located((By.LINK_TEXT, "Downloads")))
download_tab.click()

elem1=driver.find_element_by_link_text('URLs')
elem1.click()

urls = driver.find_element_by_tag_name('pre').text
urls = list(urls.split('\n'))
urls

Enter a keywordcat


['http://farm2.static.flickr.com/1369/1162408779_6eb2d41a35.jpg',
 'http://static.flickr.com/1331/536306122_c45e305163.jpg',
 'http://farm3.static.flickr.com/2072/2088622613_2660469c6c.jpg',
 'http://farm2.static.flickr.com/1245/1259825348_6a2aa94e8d.jpg',
 'http://farm1.static.flickr.com/190/497844091_17ae70e427.jpg',
 'http://static.flickr.com/121/287178609_e2da31d419.jpg',
 'http://farm1.static.flickr.com/135/331620804_e315322a82.jpg',
 'http://farm1.static.flickr.com/120/260448163_03c2748acd.jpg',
 'http://farm1.static.flickr.com/118/287178612_7f68f9563b.jpg',
 'http://farm1.static.flickr.com/45/135579916_ce5bd90475.jpg',
 'http://farm1.static.flickr.com/88/262255460_0bb183a463.jpg',
 'http://farm1.static.flickr.com/52/129880739_47ef4d064c.jpg',
 'http://farm1.static.flickr.com/194/468615076_a2450bf20b.jpg',
 'http://static.flickr.com/1138/570147673_400fd3669a.jpg',
 'http://www.ualberta.ca/~dcarney/Pictures/Tigerlily1.jpg',
 'http://static.flickr.com/1077/807830062_43db2901c8.jpg'

In [19]:
clean_urls = []
for url in urls:
    try:
        status_code = urlopen(url).getcode()
        if status_code == 200:
            clean_urls.append(url)
    except:
        pass
len(clean_urls)

1118

In [24]:
def url_to_image(url):
    req = urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1)
        
    # return the image
    return img

In [25]:
div = int(len(clean_urls)*0.80)
train = clean_urls[0:div]
test = clean_urls[div:-1]

index = 0
for url in train:
    try:
        dirname = 'train/' + keyword
        if not os.path.exists(dirname):
            os.makedirs(dirname)
        file_name = 'img-' + str(index) + '.jpg'
        try:
            img = url_to_image(url)
            cv2.imwrite(os.path.join(dirname, file_name), img)
        except:
            pass
        index = index + 1
    except:
        pass

index = 0
for url in test:
    try:
        dirname = 'test/' + keyword
        if not os.path.exists(dirname):
            os.makedirs(dirname)
        file_name = 'img-' + str(index) + '.jpg'
        try:
            img = url_to_image(url)
            cv2.imwrite(os.path.join(dirname, file_name), img)
        except:
            pass
        index = index + 1
    except:
        pass